<a href="https://colab.research.google.com/github/Klim2007/wall_mart_sales_kaggle/blob/main/WallMart_sales_validation_checking%20_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PREDICTION OF SALES BY STORES, CPI & FUEL PRICE 
(USING XGBOOST XGREGRESSOR)

In [166]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [167]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime as dt
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from xgboost import XGBRegressor
import plotly.express as px
import plotly.graph_objects as go
import datetime
import seaborn as sns

DATA PREPROCESSING

In [168]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/walmart-sales-dataset-of-45stores.csv',parse_dates=['Date'])

/usr/local/lib/python3.9/dist-packages/pandas/core/tools/datetimes.py:1063: UserWarning:

Parsing '19-02-2010' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/usr/local/lib/python3.9/dist-packages/pandas/core/tools/datetimes.py:1063: UserWarning:

Parsing '26-02-2010' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/usr/local/lib/python3.9/dist-packages/pandas/core/tools/datetimes.py:1063: UserWarning:

Parsing '19-03-2010' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/usr/local/lib/python3.9/dist-packages/pandas/core/tools/datetimes.py:1063: UserWarning:

Parsing '26-03-2010' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/usr/local/lib/python3.9/dist-packages/pandas/core/tools/datetimes.py:1063: UserWarning:

Parsing '16-04-2010' in DD/MM/YYYY format. Provide format or speci

In [169]:
df

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-05-02,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,2010-12-02,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,2010-05-03,1554806.68,0,46.50,2.625,211.350143,8.106
...,...,...,...,...,...,...,...,...
6430,45,2012-09-28,713173.95,0,64.88,3.997,192.013558,8.684
6431,45,2012-05-10,733455.07,0,64.89,3.985,192.170412,8.667
6432,45,2012-12-10,734464.36,0,54.47,4.000,192.327265,8.667
6433,45,2012-10-19,718125.53,0,56.47,3.969,192.330854,8.667


In [170]:
df.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6435.000000,6.435000e+03,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000
mean,23.000000,1.046965e+06,0.069930,60.663782,3.358607,171.578394,7.999151
std,12.988182,5.643666e+05,0.255049,18.444933,0.459020,39.356712,1.875885
min,1.000000,2.099862e+05,0.000000,-2.060000,2.472000,126.064000,3.879000
25%,12.000000,5.533501e+05,0.000000,47.460000,2.933000,131.735000,6.891000
50%,23.000000,9.607460e+05,0.000000,62.670000,3.445000,182.616521,7.874000
75%,34.000000,1.420159e+06,0.000000,74.940000,3.735000,212.743293,8.622000
max,45.000000,3.818686e+06,1.000000,100.140000,4.468000,227.232807,14.313000


In [171]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-05-02,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,2010-12-02,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,2010-05-03,1554806.68,0,46.50,2.625,211.350143,8.106


In [172]:
df.dtypes

Store                    int64
Date            datetime64[ns]
Weekly_Sales           float64
Holiday_Flag             int64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
dtype: object

In [173]:
df.value_counts()

Store  Date        Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  CPI         Unemployment
1      2010-01-10  1453329.50    0             71.89        2.603       211.671989  7.838           1
30     2012-10-19  437537.29     0             68.52        3.594       223.059808  6.170           1
31     2010-03-12  1338716.37    0             50.43        2.708       211.265543  8.163           1
       2010-03-09  1302047.48    0             82.57        2.577       211.188693  8.099           1
       2010-02-26  1344354.41    0             45.31        2.561       210.975957  8.324           1
                                                                                                   ..
15     2012-10-26  558473.60     0             55.75        4.071       138.728161  7.992           1
       2012-10-19  555652.77     0             52.89        4.153       138.833613  7.992           1
       2012-10-08  590453.63     0             73.99        3.863       138.143774  8.1

Analyzing data and dependancies
1. Store Sales Hollidays vs Working days

In [174]:
store_sales_working = df[df['Holiday_Flag'].isin([0])]
sales_stores = store_sales_working.groupby('Store').Weekly_Sales.mean().reset_index()
fig = px.bar(sales_stores, x = "Store", y ="Weekly_Sales",color="Store",title="Store weekly Sales",height=400)
fig.show()

2.Hollyday_Sales

In [175]:
store_sales_hollidays = df[df['Holiday_Flag'].isin([1])]
a = store_sales_hollidays.groupby(by="Store", dropna=False).mean().reset_index()
fig = px.bar(a, x='Store', y='Weekly_Sales',color='Store',title="Stores Overall Sales")
fig.show()  

employment working days vs hollydays

In [176]:
store_emp_hollidays = df[df['Holiday_Flag'].isin([1])]
store_emp_wrk = df[df['Holiday_Flag'].isin([0])]

store_emp_hollidays['mounth'] = store_emp_hollidays['Date'].dt.month
store_emp_wrk['mounth'] = store_emp_wrk['Date'].dt.month

<ipython-input-176-097336b97de2>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-176-097336b97de2>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [177]:
store_emp_hollidays = store_emp_hollidays.groupby('mounth')['Unemployment'].mean()
store_emp_wrk = store_emp_wrk.groupby('mounth')['Unemployment'].mean()

In [178]:
new_index = []
for i in range (1,13): new_index.append(i)
store_emp_hol = store_emp_hollidays.reindex(new_index)

In [179]:
store_emp_hol

mounth
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7     7.237333
8          NaN
9     8.097489
10    7.968456
11    8.190237
12    8.299607
Name: Unemployment, dtype: float64

In [180]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=months,
    y= store_emp_hol,
    name='unemployment on holidays',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=months,
    y= store_emp_wrk,
    name='unemployment on working days',
    marker_color='lightsalmon'
))

We see that unemployment on holidays is much higher 

CPI index review

In [181]:
CPI_stores = df.groupby('Store').CPI.mean().reset_index()
fig = px.bar(CPI_stores, x='Store', y='CPI',color='Store',title="CPI by Stores")
fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig.show()

In [182]:
CPI_hollidays = df[df['Holiday_Flag'].isin([1])]
CPI_wrk = df[df['Holiday_Flag'].isin([0])]

CPI_hollidays['mounth'] = CPI_hollidays['Date'].dt.month
CPI_wrk['mounth'] = CPI_wrk['Date'].dt.month

<ipython-input-182-a9381e709855>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-182-a9381e709855>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [183]:
CPI_hollidays.columns

Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'mounth'],
      dtype='object')

In [184]:
CPI_on_hollidays = CPI_hollidays.groupby('mounth')['CPI'].mean()
CPI_on_wrk = CPI_wrk.groupby('mounth')['CPI'].mean()

In [185]:
new_index = []
for i in range (1,13): new_index.append(i)
CPI_on_hollidays = CPI_on_hollidays.reindex(new_index)

In [186]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=months,
    y= CPI_on_hollidays,
    name='CPI on holidays',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=months,
    y= CPI_on_wrk,
    name='CPI on working days',
    marker_color='lightsalmon'
))

In [187]:
df['Year'] = df['Date'].dt.strftime('%Y')
df['Mounth'] = df['Date'].dt.strftime('%m')
df['Day'] = df['Date'].dt.strftime('%d')

In [188]:
df.head(15)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Mounth,Day
0,1,2010-05-02,1643690.90,0,42.31,2.572,211.096358,8.106,2010,05,02
1,1,2010-12-02,1641957.44,1,38.51,2.548,211.242170,8.106,2010,12,02
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010,02,19
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010,02,26
4,1,2010-05-03,1554806.68,0,46.50,2.625,211.350143,8.106,2010,05,03
5,1,2010-12-03,1439541.59,0,57.79,2.667,211.380643,8.106,2010,12,03
6,1,2010-03-19,1472515.79,0,54.58,2.720,211.215635,8.106,2010,03,19
7,1,2010-03-26,1404429.92,0,51.45,2.732,211.018042,8.106,2010,03,26
8,1,2010-02-04,1594968.28,0,62.27,2.719,210.820450,7.808,2010,02,04
9,1,2010-09-04,1545418.53,0,65.86,2.770,210.622857,7.808,2010,09,04


In [204]:
df_copi = df.loc[df['Year'] == '2010']
stores=df_copi.groupby('Store').agg({'Weekly_Sales' : 'sum'})
stores = stores.reset_index()

In [205]:
stores.head()

,Store,Weekly_Sales
0,1,73278832.00
1,2,95277864.19
2,3,18745419.00
3,4,95680470.81
4,5,14836030.77


In [211]:
str_nbr = []
for i in range(1,50):str_nbr.append(i)
fig = go.Figure()
fig.add_trace(go.Bar(
    x=str_nbr,
    y= stores['Weekly_Sales'],
    name='Weekly_Sales',
    marker_color='indianred'
))

In [ ]:
sns.boxplot(x='Year', y='Fuel_Price', data=df)


In [ ]:
sns.boxplot(x='Year', y='Unemployment', data=df)

In [ ]:
sns.boxplot(x='Year', y='CPI', data=df)

In [ ]:
numeric_features = df.drop(df.columns[[1,8,9,10]],axis = 1)

numeric_features = numeric_features.stack().reset_index().rename(
    columns={"level_1": "Features", 0: "Value"})

ax = sns.FacetGrid(data=numeric_features, col="Features",
                  col_wrap=3, sharex=False, sharey=False)
ax = ax.map(sns.distplot, "Value")

In [ ]:
numeric_features

In [ ]:
df=df.drop(['Date','Day','Year'],axis=1)

In [ ]:
# adding 3 more features
mean_month_fuel_price = df.groupby('Mounth')['Fuel_Price'].median().to_dict()
df['mean_month_fuel_price'] = df['Mounth'].map(mean_month_fuel_price)
mean_month_CPI = df.groupby('Mounth')['CPI'].median().to_dict()
df['mean_month_CPI'] = df['Mounth'].map(mean_month_CPI)
mean_month_CPI = df.groupby('Mounth')['CPI'].median().to_dict()
df['mean_month_CPI'] = df['Mounth'].map(mean_month_CPI)

In [ ]:
df

In [ ]:

df['Store'].astype('float64')
df['Mounth'].astype('float64')

In [ ]:
X = df.drop('Weekly_Sales', axis=1)
y = df['Weekly_Sales']

In [ ]:
X['Store'].astype('object')
X['Mounth'].astype('int64')

In [ ]:

X['Holiday_Flag'] = X['Holiday_Flag'].astype('object')

In [ ]:
X.dtypes

In [ ]:
numeric =list(X.dtypes[X.dtypes == float].index)
categoric = list(X.dtypes[X.dtypes == object].index)

In [ ]:
categoric
#.remove('Store')
#print(categoric.pop(0))

# Modeling

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25)

Numeric and Categoric features standartization 

Normalizing  features via logarithm 

In [ ]:
for i in X_train[numeric]:
    X_train[i] = np.log(X_train[i]+1)

In [ ]:
for i in X_test[numeric]:
    X_test[i] = np.log(X_test[i]+1)

In [ ]:
X_train[categoric].dtypes, X_train[numeric].dtypes 

In [ ]:
X_train = pd.get_dummies(X_train, columns=categoric)

In [ ]:
X_test = pd.get_dummies(X_test, columns=categoric)

In [ ]:
X_train.columns, X_test.columns

In [ ]:
import xgboost as xgb
XG = xgb.XGBRegressor(objective = 'reg:squarederror' , learning_rate = 0.1,max_depth = 10, n_estimators = 100).fit(X_train, y_train)

In [ ]:
XG.score(X_train,y_train) #0.9924816576910022

In [ ]:
y_pred_XG = XG.predict(X_test)

In [ ]:
print (y_pred_XG)

In [ ]:
print('Training Accuracy :',XG.score(X_train,y_train))
print('Testing Accuracy :',XG.score(X_test,y_test))

In [ ]:
y_test.reset_index(drop=True)

In [ ]:
fig = px.scatter(compare, x="Weekly_Sales", y="pred",color="Weekly_Sales",trendline="ols") #color='pred'
fig.show()

In [ ]:
x_pred_XG = XG.predict(X_train)
X_train['predicted_sales_next_3years'] =x_pred_XG

In [ ]:
X_train['predicted_weekly_sales']= X_train['predicted_sales_next_3years'].apply(lambda x: (x/3))

In [ ]:
X_train

In [ ]:
# check mse metric
#from sklearn.metrics import mean_squared_error 
#mse = mean_squared_error(y_test,pred)

residuals = y_test - pred
#plt.scatter(y_test,residuals)


In [ ]:
residuals